### 充电站投资值不值？Python算一把


#### 交易安排建模


In [ ]:
from absbox import API
localAPI = API("https://absbox.org/api/latest",lang='english',check=False)

from absbox import Generic
assets = [["FixedAsset"
            ,{"start":"2023-11-01","originBalance":100_0000,"originTerm":120
            ,"residual":10_0000,"period":"Monthly","amortize":"Straight"
            ,"capacity":("Fixed",24*25*120*30)}
            ,{"remainTerm":120}]]

exps = (("租金",{"type":{"recurFee":["MonthFirst",18000]}})
        ,("人员工资",{"type":{"recurFee":["MonthFirst",25000]}})
        ,("保险",{"type":{"recurFee":["MonthFirst",4500]}}))

waterfall_1 = {"amortizing":[
                ["payFee","acc01",['租金','人员工资','保险']]
            ]
            ,"cleanUp":[
                ["sellAsset",["Current|Defaulted",1.0,0],"acc01"]
                ,["payPrin","acc01",["EQ"]]
                ,["payIntResidual","acc01","EQ"]
            ]}

waterfall_2 = {"amortizing":[
                ["payFee","acc01",['租金','人员工资','保险']]
                ,["accrueAndPayInt","acc01",["A"]]
                ,["payPrin","acc01",["A"]]
            ]
            ,"cleanUp":[
                ["sellAsset",["Current|Defaulted",1.0,0],"acc01"]
                ,["payPrin","acc01",["EQ"]]
                ,["payIntResidual","acc01","EQ"]
            ]}

waterfall_3 = {"amortizing":[
                ["payFee","acc01",['租金','人员工资','保险']]
                ,["accrueAndPayInt","acc01",["A"]]
                ,["payPrin","acc01",["A"]]
                ,["payPrin","acc01",["EQ"]
                ,{"limit":{"formula":("floorWithZero"
                            ,("substract"
                                ,("accountBalance","acc01"),("constant",10000)))
                            }}]
            ]
            ,"cleanUp":[
                ["sellAsset",["Current|Defaulted",1.0,0],"acc01"]
                ,["payPrin","acc01",["EQ"]]
                ,["payIntResidual","acc01","EQ"]
            ]}

capStructure1 = (("EQ",{"balance":200_0000
                    ,"rate":0.0
                    ,"originBalance":200_0000
                    ,"originRate":0.00
                    ,"startDate":"2024-01-01"
                    ,"rateType":{"Fixed":0.00}
                    ,"bondType":{"Equity":None}}),
                )


capStructure2 = (("A",{"balance":100_0000
                    ,"rate":0.06
                    ,"originBalance":100_0000
                    ,"originRate":0.00
                    ,"startDate":"2024-01-01"
                    ,"rateType":{"Fixed":0.06}
                    ,"bondType":{"Sequential":None}})
                ,("EQ",{"balance":100_0000
                    ,"rate":0.0
                    ,"originBalance":100_0000
                    ,"originRate":0.00
                    ,"startDate":"2024-01-01"
                    ,"rateType":{"Fixed":0.00}
                    ,"bondType":{"Equity":None}}),
                )


evStation = Generic(
    "TEST01"
    ,{"cutoff":"2024-01-01","closing":"2024-01-01","firstPay":"2024-02-01"
    ,"payFreq":"MonthEnd","poolFreq":"MonthEnd","stated":"2050-01-01"}
    ,{'assets':assets}
    ,(("acc01",{"balance":0}),)
    ,capStructure1
    ,exps
    ,waterfall_1
    ,[["CollectedCash","acc01"]]
    ,None
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
    )

myAssump = ("Pool"
            ,("Fixed",[["2024-01-01",0.05]
                    ,["2025-01-01",0.08]
                    ,["2026-01-01",0.1]]
                    ,[["2024-01-01",0.3]
                    ,["2025-01-01",0.35]
                    ,["2026-01-01",0.4]])
            ,None
            ,None)

p = localAPI.run(evStation,poolAssump=myAssump
                    ,runAssump=[("call",{"afterDate":"2034-07-01"})]
                    ,read=True)

from absbox.local.analytics import irr
irr(p['bonds']['EQ'],init=('2024-01-01',-200_0000))

#### 引入优先级

evStation = Generic(
    "TEST01"
    ,{"cutoff":"2024-01-01","closing":"2024-01-01","firstPay":"2024-02-01"
    ,"payFreq":"MonthEnd","poolFreq":"MonthEnd","stated":"2050-01-01"}
    ,{'assets':assets}
    ,(("acc01",{"balance":0}),)
    ,capStructure2
    ,exps
    ,waterfall_2
    ,[["CollectedCash","acc01"]]
    ,None
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
    )

myAssump = ("Pool"
            ,("Fixed",[["2024-01-01",0.05]
                    ,["2025-01-01",0.08]
                    ,["2026-01-01",0.1]]
                    ,[["2024-01-01",0.3]
                    ,["2025-01-01",0.35]
                    ,["2026-01-01",0.4]])
            ,None
            ,None)

p = localAPI.run(evStation,poolAssump=myAssump
                    ,runAssump=[("call",{"afterDate":"2034-07-01"})]
                    ,read=True)

irr(p['bonds']['EQ'],init=('2024-01-01',-100_0000))

#### 期间分配

In [ ]:
evStation = Generic(
    "TEST01"
    ,{"cutoff":"2024-01-01","closing":"2024-01-01","firstPay":"2024-02-01"
    ,"payFreq":"MonthEnd","poolFreq":"MonthEnd","stated":"2050-01-01"}
    ,{'assets':assets}
    ,(("acc01",{"balance":0}),)
    ,capStructure2
    ,exps
    ,waterfall_3
    ,[["CollectedCash","acc01"]]
    ,None
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
    )

myAssump = ("Pool"
            ,("Fixed",[["2024-01-01",0.05]
                    ,["2025-01-01",0.08]
                    ,["2026-01-01",0.1]]
                    ,[["2024-01-01",0.3]
                    ,["2025-01-01",0.35]
                    ,["2026-01-01",0.4]])
            ,None
            ,None)

p = localAPI.run(evStation,poolAssump=myAssump
                    ,runAssump=[("call",{"afterDate":"2034-07-01"})]
                    ,read=True)

irr(p['bonds']['EQ'],init=('2024-01-01',-100_0000))

#### 敏感性分析

In [ ]:
scenarioMap = {
    "base":("Pool"
            ,("Fixed",[["2024-01-01",0.05]
                    ,["2025-01-01",0.08]
                    ,["2026-01-01",0.1]]
                    ,[["2024-01-01",0.3]
                    ,["2025-01-01",0.35]
                    ,["2026-01-01",0.4]])
            ,None, None)
    ,"lowUtil" : ("Pool"
            ,("Fixed",[["2024-01-01",0.05]
                    ,["2025-01-01",0.05]
                    ,["2026-01-01",0.8]]
                    ,[["2024-01-01",0.3]
                    ,["2025-01-01",0.35]
                    ,["2026-01-01",0.4]])
            ,None, None)
    ,"lowPrice" : ("Pool"
            ,("Fixed",[["2024-01-01",0.05]
                    ,["2025-01-01",0.05]
                    ,["2026-01-01",0.8]]
                    ,[["2024-01-01",0.25]
                    ,["2025-01-01",0.30]
                    ,["2026-01-01",0.35]])
            ,None, None)
}
p = localAPI.run(evStation,poolAssump=scenarioMap
                    ,runAssump=[("call",{"afterDate":"2034-07-01"})]
                    ,read=True)

from absbox.local.analytics import irr
{k:irr(v['bonds']['EQ'],init=('2024-01-01',-100_0000))
for k,v in p.items()}